In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append(os.path.abspath(".."))

In [2]:
import numpy as np
import pandas as pd

In [ ]:
result = pd.read_csv("../experiments/results/result_rchar_1709153316/gpt-4-turbo_2/results.csv", index_col=0)

In [20]:
result.to_csv("../experiments/results/result_rchar_1709153316/gpt-4-turbo_2/results_1.csv")

## Separate Response - based on type of intervention

In [8]:
import glob
import numpy as np
import pandas as pd

from rbias.constants import DATASETS, C_LADDERS, TASKS
from rbias.query import analyze_result
from rbias.process import sep_responses_based_on_inter
from rbias.utils import get_taskwise_filters

In [23]:
datasets = DATASETS
c_rungs = C_LADDERS

model_name = "gpt-3.5-turbo"
#model_name = "gpt-4-turbo"
#model_name = "gpt-4"
#model_name = "llama-2-7b"

In [24]:
response_folders = [f"../experiments/results/result_rchar_1710342923/{model_name}_1/",
         f"../experiments/results/result_cs_1710342947/{model_name}_1/",
         f"../experiments/results/result_adv_1710343186/{model_name}_1/"]



In [25]:
def sep_responses_based_on_inter_task_0(response_folders, filters):
    print("Separating responses according to obvious and non-obvious intervenstions...")
    for folder_path in response_folders:
        print("result_folder: ", folder_path)
        obv_res = []
        non_obv_res = []
        for rung in C_LADDERS:
            for task_id in range(1):
                exp_name = f"{rung}_{task_id}"
                print("Exp name:", exp_name)
                sub_folder_path = os.path.join(folder_path, exp_name)
                files = sorted(glob.glob(f"{sub_folder_path}/*.csv"))
                for i, file in enumerate(files):
                    df = pd.read_csv(file, index_col=0, sep = "|")
                    # separate them into two dfs - one that change and the other that doesn't
                    obv_inter, non_obv_inter = df.loc[filters[task_id][0]], df.loc[filters[task_id][1]]
                    
                    # store them in separate folders
                    obv_path = os.path.join(sub_folder_path, "obv_0")
                    if not os.path.isdir(obv_path):
                        os.mkdir(obv_path)
                    non_obv_path = os.path.join(sub_folder_path, "non_obv_0")
                    if not os.path.isdir(non_obv_path):
                        os.mkdir(non_obv_path)
                    obv_inter.to_csv(os.path.join(obv_path, f"response_{i}.csv"), sep="|")
                    non_obv_inter.to_csv(os.path.join(non_obv_path, f"response_{i}.csv"), sep="|")
                    # generate separate results for obv and non_obv
                    g_res = analyze_result(obv_inter)
                    b_res = analyze_result(non_obv_inter)
                    obv_res.append([exp_name, i, g_res[-1], *(g_res[:-1])])
                    non_obv_res.append([exp_name, i, b_res[-1], *(b_res[:-1])])         
        # store the result files
        obv_res_df = pd.DataFrame(obv_res, columns = ["exp_name", "trial","size", "acc", "r_acc", "yes_acc", "no_acc", "f1_score"])
        non_obv_res_df = pd.DataFrame(non_obv_res, columns = ["exp_name", "trial", "size", "acc", "r_acc", "yes_acc", "no_acc", "f1_score"])
        obv_res_df.to_csv(os.path.join(folder_path, "obv_results_0.csv"))
        non_obv_res_df.to_csv(os.path.join(folder_path, "non_obv_results_0.csv"))
    print("Done!")

In [26]:
filters = get_taskwise_filters()
sep_responses_based_on_inter(response_folders, filters)

[1] [0]
[1 3 5] [0 2 4]
[0 1 2 4 5 7 8] [3 6]
[ 0  1  2  4  5  7  8  9 10 11 13 14 16 17 18 19 20 22 23 25 26] [ 3  6 12 15 21 24]
[0 1 2 4 6] [3 5 7 8]
[ 0  1  2  4  6  9 10 11 13 15 18 19 20 22 24] [ 3  5  7  8 12 14 16 17 21 23 25 26]
Separating responses according to obvious and non-obvious intervenstions...
result_folder:  ../experiments/results/result_rchar_1710342923/gpt-3.5-turbo_1/
Exp name: obs_0
Exp name: inter_0
Exp name: sub_0
result_folder:  ../experiments/results/result_cs_1710342947/gpt-3.5-turbo_1/
Exp name: obs_0
Exp name: inter_0
Exp name: sub_0
result_folder:  ../experiments/results/result_adv_1710343186/gpt-3.5-turbo_1/
Exp name: obs_0
Exp name: inter_0
Exp name: sub_0
Done!


In [ ]:
from rbias.process import prep_taskwise_across_data_rungs, prep_taskwise_obv_non_obv_inter

#### Script to generate model wise diff comparisons

In [ ]:
models = ["gpt-3.5-turbo", "gpt-4", "gpt-4-turbo", "llama-2-7b"]
res_file_template = ["../experiments/results/result_rchar_1710342923/{model_name}_1/results.csv",
         "../experiments/results/result_cs_1710342947/{model_name}_1/results.csv",
         "../experiments/results/result_adv_1710343186/{model_name}_1/results.csv"]

In [ ]:
cs_adv_diff = []
cs_rchar_diff = []
cs_adv_std = []
cs_rchar_std = []
for model_name in models:
    result_files = [ f.format(model_name=model_name) for f in res_file_template ]
    _, res_adata = prep_taskwise_across_data_rungs(result_files)
    oi_avg = res_adata[:, :, 0, 0] - res_adata[:, :, 1, 0]
    oi_std_max = np.maximum(res_adata[:, :, 0, 1], res_adata[:, :, 1, 1])
    oi_std_sum = res_adata[:, :, 0, 1] + res_adata[:, :, 1, 1]
    # append oi_diff of cs and adv.
    cs_adv_diff.append([oi_avg[:, 1], oi_avg[:, 2]])
    cs_rchar_diff.append([oi_avg[:, 1], oi_avg[:, 0]])
    cs_adv_std.append([oi_std_sum[:, 1], oi_std_sum[:, 2]])
    cs_rchar_std.append([oi_std_sum[:, 1], oi_std_sum[:, 0]])

cs_adv_diff, cs_rchar_diff = np.array(cs_adv_diff), np.array(cs_rchar_diff)
cs_adv_std, cs_rchar_std = np.array(cs_adv_std), np.array(cs_rchar_std)


In [ ]:
cs_adv_diff

array([[[ 0.23333333,  0.15555556,  0.35555556],
        [ 0.1       ,  0.23703704,  0.35925926]],

       [[ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ]],

       [[ 0.03333333,  0.07407407,  0.11111111],
        [ 0.        ,  0.17777778,  0.15555556]],

       [[ 0.13333333,  0.01851852,  0.28888889],
        [ 0.23333333, -0.04444444,  0.12592593]]])

In [ ]:
 cs_adv_std

array([[[0.2       , 0.05443311, 0.04444444],
        [0.13333333, 0.05023948, 0.0456465 ]],

       [[0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ]],

       [[0.06666667, 0.03312693, 0.03312693],
        [0.        , 0.09591002, 0.11626192]],

       [[0.12472191, 0.12091846, 0.07216043],
        [0.13333333, 0.09798882, 0.08406274]]])

In [ ]:
cs_adv_diff[:, 0] - cs_adv_diff[:, 1]

array([[ 0.13333333, -0.08148148, -0.0037037 ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.03333333, -0.1037037 , -0.04444444],
       [-0.1       ,  0.06296296,  0.16296296]])

In [ ]:
cs_rchar_diff

array([[[0.23333333, 0.15555556, 0.35555556],
        [0.16666667, 0.28888889, 0.40740741]],

       [[0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.0962963 ]],

       [[0.03333333, 0.07407407, 0.11111111],
        [0.03333333, 0.04444444, 0.07407407]],

       [[0.13333333, 0.01851852, 0.28888889],
        [0.36666667, 0.15555556, 0.34074074]]])

In [ ]:
 cs_rchar_std

array([[[0.2       , 0.05443311, 0.04444444],
        [0.10540926, 0.04319224, 0.05737753]],

       [[0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.03777051]],

       [[0.06666667, 0.03312693, 0.03312693],
        [0.06666667, 0.03628874, 0.02342428]],

       [[0.12472191, 0.12091846, 0.07216043],
        [0.12472191, 0.05493253, 0.06907078]]])

In [ ]:
cs_rchar_diff[:, 0] - cs_rchar_diff[:, 1]

array([[ 0.06666667, -0.13333333, -0.05185185],
       [ 0.        ,  0.        , -0.0962963 ],
       [ 0.        ,  0.02962963,  0.03703704],
       [-0.23333333, -0.13703704, -0.05185185]])

#### Script to generate model wise diff comparisons for obvious interventions

In [ ]:
from rbias.process import prep_taskwise_obv_non_obv_inter

In [34]:
def prep_taskwise_obv_non_obv_inter_0(res_folder_paths):
    obv_data = []
    for f, d in zip(res_folder_paths, DATASETS):
        f = os.path.join(f, "obv_results_0.csv")
        df = pd.read_csv(f, index_col=0)
        arr = df["acc"].to_numpy()
        arr = arr.reshape((3, 1, -1))
        avg, std = np.mean(arr, axis=2), np.std(arr, axis=2)
        sub_arr = np.concatenate((avg[..., None], std[..., None]), axis = 2)
        obv_data.append(sub_arr)

    non_obv_data = []
    for f, d in zip(res_folder_paths, DATASETS):
        f = os.path.join(f, "non_obv_results_0.csv")
        df = pd.read_csv(f, index_col=0)
        arr = df["acc"].to_numpy()
        arr = arr.reshape((3, 1, -1))
        avg, std = np.mean(arr, axis=2), np.std(arr, axis=2)
        sub_arr = np.concatenate((avg[..., None], std[..., None]), axis = 2)
        non_obv_data.append(sub_arr)
    data = [obv_data, non_obv_data]
    # (obv/non) - dataset - (inter/sub) - tasks - (avg/std) -> (inter/sub) - (tasks) - (dataset) - (obv/non) - (avg/std)
    data = np.array(data).transpose([2, 3, 1, 0, 4])
    
    return data

In [35]:
models = ["gpt-3.5-turbo", "gpt-4", "gpt-4-turbo", "llama-2-7b"]
res_file_template = ["../experiments/results/result_rchar_1710342923/{model_name}_1/",
         "../experiments/results/result_cs_1710342947/{model_name}_1/",
         "../experiments/results/result_adv_1710343186/{model_name}_1/"]

In [46]:
obv_cs_adv_diff = []
obv_cs_rchar_diff = []
obv_cs_adv_std = []
obv_cs_rchar_std = []
for model_name in models:
    result_files = [ f.format(model_name=model_name) for f in res_file_template ]
    inter_res = prep_taskwise_obv_non_obv_inter_0(result_files)
    inter_res = inter_res[:, :, :, 0, :].transpose([1, 2, 0, 3])
    print(model_name, inter_res)
    oi_avg = inter_res[:, :, 0, 0] - inter_res[:, :, 1, 0]
    oi_std_max = np.maximum(inter_res[:, :, 0, 1], inter_res[:, :, 1, 1])
    oi_std_sum = inter_res[:, :, 0, 1] + inter_res[:, :, 1, 1]
    obv_cs_adv_diff.append([oi_avg[:, 1], oi_avg[:, 2]])
    obv_cs_rchar_diff.append([oi_avg[:, 1], oi_avg[:, 0]])
    obv_cs_adv_std.append([oi_std_sum[:, 1], oi_std_sum[:, 2]])
    obv_cs_rchar_std.append([oi_std_sum[:, 1], oi_std_sum[:, 0]])


obv_cs_adv_diff, obv_cs_rchar_diff = np.array(obv_cs_adv_diff), np.array(obv_cs_rchar_diff)
obv_cs_adv_std, obv_cs_rchar_std = np.array(obv_cs_adv_std), np.array(obv_cs_rchar_std)


gpt-3.5-turbo [[[[1.         0.        ]
   [0.73333333 0.24944383]
   [0.33333333 0.        ]]

  [[1.         0.        ]
   [0.8        0.26666667]
   [0.33333333 0.        ]]

  [[1.         0.        ]
   [0.86666667 0.26666667]
   [0.33333333 0.        ]]]]
gpt-4 [[[[1.         0.        ]
   [1.         0.        ]
   [1.         0.        ]]

  [[1.         0.        ]
   [1.         0.        ]
   [0.93333333 0.13333333]]

  [[1.         0.        ]
   [1.         0.        ]
   [1.         0.        ]]]]
gpt-4-turbo [[[[1.         0.        ]
   [0.93333333 0.13333333]
   [0.93333333 0.13333333]]

  [[1.         0.        ]
   [0.93333333 0.13333333]
   [0.73333333 0.13333333]]

  [[1.         0.        ]
   [1.         0.        ]
   [0.53333333 0.16329932]]]]
llama-2-7b [[[[0.86666667 0.16329932]
   [0.66666667 0.        ]
   [0.66666667 0.        ]]

  [[0.46666667 0.16329932]
   [0.66666667 0.        ]
   [0.66666667 0.        ]]

  [[0.66666667 0.21081851]
   [0.66666667

In [38]:
obv_cs_adv_diff

array([[[ 0.2       ],
        [ 0.13333333]],

       [[ 0.        ],
        [ 0.        ]],

       [[ 0.06666667],
        [ 0.        ]],

       [[-0.2       ],
        [ 0.        ]]])

In [39]:
obv_cs_adv_std

array([[[0.26666667],
        [0.26666667]],

       [[0.        ],
        [0.        ]],

       [[0.13333333],
        [0.        ]],

       [[0.16329932],
        [0.21081851]]])

In [40]:
obv_cs_adv_diff[:, 0] - obv_cs_adv_diff[:, 1]

array([[ 0.06666667],
       [ 0.        ],
       [ 0.06666667],
       [-0.2       ]])

In [47]:
obv_cs_rchar_diff

array([[[ 0.2       ],
        [ 0.26666667]],

       [[ 0.        ],
        [ 0.        ]],

       [[ 0.06666667],
        [ 0.06666667]],

       [[-0.2       ],
        [ 0.2       ]]])

In [48]:
obv_cs_rchar_std

array([[[0.26666667],
        [0.24944383]],

       [[0.        ],
        [0.        ]],

       [[0.13333333],
        [0.13333333]],

       [[0.16329932],
        [0.16329932]]])

In [49]:
obv_cs_rchar_diff[:, 0] - obv_cs_rchar_diff[:, 1]

array([[-0.06666667],
       [ 0.        ],
       [ 0.        ],
       [-0.4       ]])

### Script to generate retrieval performance

In [6]:
models = ["gpt-3.5-turbo", "gpt-4", "gpt-4-turbo", "llama-2-7b"]
res_file_template = ["../experiments/results/result_rchar_1711052432/{model_name}_1/results.csv",
         "../experiments/results/result_cs_1711052444/{model_name}_1/results.csv",
         "../experiments/results/result_adv_1711052457/{model_name}_1/results.csv"]

In [22]:
res = []
for model_name in models:
    res_files = [ f.format(model_name=model_name) for f in res_file_template[:1] ]
    df = pd.read_csv(res_files[0], index_col=0)
    obs_perf = df.iloc[list(range(15))]["acc"].to_numpy().reshape(3, -1)
    m, sterr = obs_perf.mean(axis=1), obs_perf.std(axis=1)/np.sqrt(5)
    res.append(list(zip(m.tolist(), sterr.tolist())))
    # res.append((model_name, m, sterr))
print(res)
res_df = pd.DataFrame(res, columns = [ "Graph 1", "Graph 2", "Graph 3"], index = models).map(lambda x: (round(x[0], 2), round(x[1], 2)))
res_df



[[(1.0, 0.0), (1.0, 0.0), (0.9888888888888889, 0.009938079899999069)], [(1.0, 0.0), (1.0, 0.0), (1.0, 0.0)], [(1.0, 0.0), (1.0, 0.0), (1.0, 0.0)], [(0.8666666666666666, 0.0557773351022717), (0.6333333333333333, 0.029814239699997195), (0.8555555555555555, 0.012171612389003673)]]


,Graph 1,Graph 2,Graph 3
gpt-3.5-turbo,"(1.0, 0.0)","(1.0, 0.0)","(0.99, 0.01)"
gpt-4,"(1.0, 0.0)","(1.0, 0.0)","(1.0, 0.0)"
gpt-4-turbo,"(1.0, 0.0)","(1.0, 0.0)","(1.0, 0.0)"
llama-2-7b,"(0.87, 0.06)","(0.63, 0.03)","(0.86, 0.01)"


In [24]:
for row in res_df.iterrows():
    print(" & ".join(map(lambda x: f"${x[0]} \\pm {x[1]}$", row[1].values)) + " \\\\")

$100.0 \pm 0.0$ & $100.0 \pm 0.0$ & $99.0 \pm 1.0$ \\
$100.0 \pm 0.0$ & $100.0 \pm 0.0$ & $100.0 \pm 0.0$ \\
$100.0 \pm 0.0$ & $100.0 \pm 0.0$ & $100.0 \pm 0.0$ \\
$87.0 \pm 6.0$ & $63.0 \pm 3.0$ & $86.0 \pm 1.0$ \\
